## Implementation of Transformer for machine translation
### The __torchtext.data__ may through an error stating `no module found named "Field"` which probably arises due to deprecation of this module in the newer version of torch. Execute the cell below to install the `torchtext version 0.6.0` to run the notebook. This is because the _Field_ and _TabularDataset_ makes the vocabulary and dataloader creation much simpler.
```python
pip install torchtext==0.6.0
print(torchtext.__version__)
```
You also may need to restart the runtime

In [2]:
# !pip install torchtext==0.6.0
# print(torchtext.__version__)

In [23]:
import torch
import torchtext
from torchtext import vocab
import torch.nn as nn
import torch.nn.functional as F
from torchtext.data import Field, BucketIterator, TabularDataset
from torch import Tensor
from torch.nn import TransformerEncoder, TransformerDecoder,TransformerEncoderLayer, TransformerDecoderLayer
import torch.optim as optim
import os
import csv, json
import pickle
import random
from collections import Counter
import warnings
import re, string
import math
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from string import digits
warnings.filterwarnings("ignore")

In [4]:
# replace the name of th test data after -O
!wget 'https://drive.google.com/uc?id=19WHTYU0j3EBFTNR_OldP3c6AKNqonH52' -O test_data.json


--2023-10-04 11:45:47--  https://drive.google.com/uc?id=19WHTYU0j3EBFTNR_OldP3c6AKNqonH52
Resolving drive.google.com (drive.google.com)... 209.85.200.102, 209.85.200.100, 209.85.200.138, ...
Connecting to drive.google.com (drive.google.com)|209.85.200.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0c-54-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/66c79s9a5dtsnq5tripdefbn4764eo5o/1696419900000/13631053167411324863/*/19WHTYU0j3EBFTNR_OldP3c6AKNqonH52?uuid=ab43cd93-2aa0-49df-bce2-189a6a9a7813 [following]
--2023-10-04 11:45:48--  https://doc-0c-54-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/66c79s9a5dtsnq5tripdefbn4764eo5o/1696419900000/13631053167411324863/*/19WHTYU0j3EBFTNR_OldP3c6AKNqonH52?uuid=ab43cd93-2aa0-49df-bce2-189a6a9a7813
Resolving doc-0c-54-docs.googleusercontent.com (doc-0c-54-docs.googleusercontent.com)... 173.194.195.132, 2607:f8b0:4001:c1a::84
Connecting to doc-0

In [5]:
TEST_FILE = "test_data.json"

val_ids = []

def fetchtestData():

    DATA_DIR = "testData"
    if not os.path.exists(DATA_DIR):
        os.makedirs(DATA_DIR)

    with open(TEST_FILE, "r") as file:
        val_data = json.load(file)
        for lang_type, lang_data in val_data.items():
            print(lang_type)
            for data_type, data_entries in lang_data.items():
                ids, en_sns = [], []
                for ent_id, ent_data in data_entries.items():
                    val_ids.append(ent_id)
                    ids.append(ent_id)
                    en_sns.append(ent_data["source"])

                valdf = pd.DataFrame({"id" : ids, "english" : en_sns})
                valdf.to_csv(os.path.join(DATA_DIR,"test{}.csv".format(lang_type)))
    print(len(val_ids))

def merge():
    DATA_DIR = "models/Translations"
    RESULTS = "answer.csv"

    if os.path.exists(os.path.join(DATA_DIR, RESULTS)):
        os.remove(os.path.join(DATA_DIR,RESULTS))
    val_ids = []

    with open(VALIDATION_FILE, "r") as file:
        val_data = json.load(file)
        for lang_type, lang_data in val_data.items():
            print(lang_type)
            for data_type, data_entries in lang_data.items():
                ids, en_sns = [], []
                for ent_id, ent_data in data_entries.items():
                    val_ids.append(int(ent_id))

    print(len(val_ids))
    # print(val_ids)

    # Reading all the dataframes
    files = os.listdir(DATA_DIR)
    files = [file for file in files if file.endswith(".csv")]
    print(files)
    data = [pd.read_csv(os.path.join(DATA_DIR, file)) for file in files]
    res = pd.concat(data, axis=0, ignore_index=False)
    res.drop(["Unnamed: 0", "english"], axis=1, inplace=True)
    res = res.rename(columns={"id": "ID", "translated": "Translated"})
    res_sorted = res[res["ID"].isin(val_ids)].sort_values(by="ID")
    res_sorted.to_csv(os.path.join(DATA_DIR, RESULTS), sep="\t", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"', escapechar='\\')

fetchtestData()

English-Bengali
English-Gujarati
English-Hindi
English-Kannada
English-Malayalam
English-Tamil
English-Telgu
114647


In [6]:
import os
import gdown

# Direct download URLs
hin = 'https://drive.google.com/uc?export=download&id=1acr5l2kPbksgMmBYutiT1NBPLNyKiNH6'
tam = 'https://drive.google.com/uc?export=download&id=1tmbFLO8EkYK_xWQTGeh3j6cM4dDWSaz5'
ben = 'https://drive.google.com/uc?export=download&id=14ZQdqI_G8wS8r9_ha3Kr7QJV9bJiMcNT'
kan = 'https://drive.google.com/uc?export=download&id=15qzjTmcHG26CvmsOSPSK2BrkLhcdZjde'
guj = 'https://drive.google.com/uc?export=download&id=1k56Imci4OEGi3dHUJeAqTGT0qa15EQwV'
tel = 'https://drive.google.com/uc?export=download&id=1OiHVj-HtWHkX86oQ-1drlAWRy7rlk9rI'
mal = 'https://drive.google.com/uc?export=download&id=1sHBCkckoc52EPL_3iQfTs11YqP_0DKAG'

names = ['hindi', 'tamil', 'bengali', 'kannada', 'gujarati', 'telgu', 'malayalam']
nbs = [hin, tam, ben, kan, guj, tel, mal]

for index, nm in enumerate(names):
    file_url = nbs[index]
    output_file = f"{nm}.pth"
    # Use gdown to download the file
    gdown.download(file_url, output_file, quiet=False)

# Verify the download
print("Download complete. Files in the current directory:")
print(os.listdir())


Downloading...
From: https://drive.google.com/uc?export=download&id=1acr5l2kPbksgMmBYutiT1NBPLNyKiNH6
To: /content/hindi.pth
100%|██████████| 368M/368M [00:07<00:00, 48.3MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1tmbFLO8EkYK_xWQTGeh3j6cM4dDWSaz5
To: /content/tamil.pth
100%|██████████| 354M/354M [00:07<00:00, 47.0MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=14ZQdqI_G8wS8r9_ha3Kr7QJV9bJiMcNT
To: /content/bengali.pth
100%|██████████| 365M/365M [00:09<00:00, 39.4MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=15qzjTmcHG26CvmsOSPSK2BrkLhcdZjde
To: /content/kannada.pth
100%|██████████| 336M/336M [00:05<00:00, 56.6MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1k56Imci4OEGi3dHUJeAqTGT0qa15EQwV
To: /content/gujarati.pth
100%|██████████| 345M/345M [00:07<00:00, 45.9MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1OiHVj-HtWHkX86oQ-1drlAWRy7rlk9rI
To: /conten

Download complete. Files in the current directory:
['.config', 'malayalam.pth', 'bengali.pth', 'gujarati.pth', 'kannada.pth', 'tamil.pth', 'test_data.json', 'hindi.pth', 'testData', 'telgu.pth', 'sample_data']


In [7]:
import os
import gdown

# Direct download URLs
hin_vocab = "https://drive.google.com/uc?export=download&id=1puzGRvWkZjruQ_gx60nJj2I5e4J80vu6"
tam_vocab = "https://drive.google.com/uc?export=download&id=1V5C4lhBpqV5xkN06FGAkZSE4qj3gj_LA"
ben_vocab = "https://drive.google.com/uc?export=download&id=1-D2zoKuXOnZdnTg9Q9hGPWsrgvX6TP18"
kan_vocab = "https://drive.google.com/uc?export=download&id=1cjIZKA_56Eo0RzgNbEVdUFjSLGzSVq99"
guj_vocab = "https://drive.google.com/uc?export=download&id=1mgNQ991u1wF0hya9oEYJuCU9eSaq5sWe"
tel_vocab = "https://drive.google.com/uc?export=download&id=1dTB9m-69Se5ajbH7pDFstwBKSfBnd49d"
mal_vocab = "https://drive.google.com/uc?export=download&id=1MQ5u-hb_9TcaswKu5IEqrtDGdUrByo-D"

vocab_names = ['hindi', 'tamil', 'bengali', 'kannada', 'gujarati', 'telgu', 'malayalam']
nbs_vocabs = [hin_vocab, tam_vocab, ben_vocab, kan_vocab, guj_vocab, tel_vocab, mal_vocab]

for index, nm in enumerate(vocab_names):
    file_url = nbs_vocabs[index]
    output_file = f"{nm}.pkl"
    # Use gdown to download the file
    gdown.download(file_url, output_file, quiet=False)

# Verify the download
print("Download complete. Files in the current directory:")
print(os.listdir())


Downloading...
From: https://drive.google.com/uc?export=download&id=1puzGRvWkZjruQ_gx60nJj2I5e4J80vu6
To: /content/hindi.pkl
100%|██████████| 2.27M/2.27M [00:00<00:00, 224MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1V5C4lhBpqV5xkN06FGAkZSE4qj3gj_LA
To: /content/tamil.pkl
100%|██████████| 4.74M/4.74M [00:00<00:00, 21.5MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1-D2zoKuXOnZdnTg9Q9hGPWsrgvX6TP18
To: /content/bengali.pkl
100%|██████████| 3.08M/3.08M [00:00<00:00, 235MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1cjIZKA_56Eo0RzgNbEVdUFjSLGzSVq99
To: /content/kannada.pkl
100%|██████████| 3.29M/3.29M [00:00<00:00, 170MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1mgNQ991u1wF0hya9oEYJuCU9eSaq5sWe
To: /content/gujarati.pkl
100%|██████████| 2.60M/2.60M [00:00<00:00, 207MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1dTB9m-69Se5ajbH7pDFstwBKSfBnd49d
To: /

Download complete. Files in the current directory:
['.config', 'malayalam.pth', 'bengali.pkl', 'bengali.pth', 'gujarati.pth', 'malayalam.pkl', 'kannada.pkl', 'hindi.pkl', 'gujarati.pkl', 'kannada.pth', 'telgu.pkl', 'tamil.pth', 'test_data.json', 'hindi.pth', 'testData', 'tamil.pkl', 'telgu.pth', 'sample_data']


In [8]:
import os
import gdown

# Adjusted URLs for direct downloading
eng_hindi_vocab = "https://drive.google.com/uc?export=download&id=1cwjYFublDl2PncWO-z1R0CFL3iKVguyj"
eng_tamil_vocab = "https://drive.google.com/uc?export=download&id=1zbWJ9t2ufxsA_sicK5bZGFSIuHX7S_FV"
eng_bengali_vocab = "https://drive.google.com/uc?export=download&id=11EvhUzf-aPfeEagI7DqUBWz9yv4NR2b-"
eng_kannada_vocab = "https://drive.google.com/uc?export=download&id=1FO8TUY7CiK7eB63GyR0FRb0SN4ZzmXex"
eng_gujarati_vocab = "https://drive.google.com/uc?export=download&id=1CQH1Dc26gkmYcoO4HfvMGYQdmHtcf45d"
eng_telgu_vocab = "https://drive.google.com/uc?export=download&id=18g2AvdeDN110661OUVf6LP9NbCQn21kI"
eng_malayalam_vocab = "https://drive.google.com/uc?export=download&id=1ZnTc_FwFyn2CjMB5baqPiYnNDHbl4_eH"

vocab_names = ['hindi', 'tamil', 'bengali', 'kannada', 'gujarati', 'telgu', 'malayalam']
nbs_vocabs = [eng_hindi_vocab, eng_tamil_vocab, eng_bengali_vocab, eng_kannada_vocab, eng_gujarati_vocab, eng_telgu_vocab, eng_malayalam_vocab]

for index, nm in enumerate(vocab_names):
    file_url = nbs_vocabs[index]
    output_file = "eng_{}.pkl".format(nm)
    # Use gdown to download the file
    gdown.download(file_url, output_file, quiet=False)

# Verify the download
print("Download complete. Files in the current directory:")
print(os.listdir())


Downloading...
From: https://drive.google.com/uc?export=download&id=1cwjYFublDl2PncWO-z1R0CFL3iKVguyj
To: /content/eng_hindi.pkl
100%|██████████| 1.29M/1.29M [00:00<00:00, 146MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1zbWJ9t2ufxsA_sicK5bZGFSIuHX7S_FV
To: /content/eng_tamil.pkl
100%|██████████| 1.09M/1.09M [00:00<00:00, 126MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=11EvhUzf-aPfeEagI7DqUBWz9yv4NR2b-
To: /content/eng_bengali.pkl
100%|██████████| 1.23M/1.23M [00:00<00:00, 171MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1FO8TUY7CiK7eB63GyR0FRb0SN4ZzmXex
To: /content/eng_kannada.pkl
100%|██████████| 864k/864k [00:00<00:00, 133MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=1CQH1Dc26gkmYcoO4HfvMGYQdmHtcf45d
To: /content/eng_gujarati.pkl
100%|██████████| 982k/982k [00:00<00:00, 115MB/s]
Downloading...
From: https://drive.google.com/uc?export=download&id=18g2AvdeDN110661OUVf6LP9

Download complete. Files in the current directory:
['.config', 'malayalam.pth', 'bengali.pkl', 'bengali.pth', 'gujarati.pth', 'eng_malayalam.pkl', 'eng_telgu.pkl', 'eng_tamil.pkl', 'malayalam.pkl', 'kannada.pkl', 'hindi.pkl', 'gujarati.pkl', 'eng_bengali.pkl', 'kannada.pth', 'telgu.pkl', 'tamil.pth', 'eng_kannada.pkl', 'eng_hindi.pkl', 'eng_gujarati.pkl', 'test_data.json', 'hindi.pth', 'testData', 'tamil.pkl', 'telgu.pth', 'sample_data']


In [9]:
# !wget https://drive.google.com/drive/folders/1PgGeWF6Rp-0Z30LBn6-70Cl-Vz_uxms5?usp=sharing -O models

In [12]:
# Set the dataset name
l = ["Bengali", "Gujarati", "Hindi", "Malayalam", "Telgu", "Kannada", "Tamil"]

for e in names:
# Read the CSV file from the specified directory into a DataFrame
  data = pd.read_csv("/content/testData/testEnglish-{}.csv".format(e.title()))

  # Drop the unnecessary columns "Unnamed: 0" and "entry_id" from the DataFrame
  data.drop(["Unnamed: 0", "id"], inplace=True, axis=1)

  # Note: The next operation seems redundant as "entry_id" has already been dropped.
  # Rename the column "entry_id" to "id" (if it exists)
  data = data.rename(columns={"id": "id"})

  # Display the first 10 rows of the cleaned DataFrame
  # (This will be visible in interactive environments like Jupyter Notebook)
  data.head(10)

  # Write the cleaned data back to a new CSV file in the current directory
  data.to_csv("{}.csv".format(e), index=False)


In [13]:
def preprocess(text):
    """
    Convert all the text into lower letters
    Remove the words betweent brakets ()
    Remove these characters: {'$', ')', '?', '"', '’', '.',  '°', '!', ';', '/', "'", '€', '%', ':', ',', '('}
    Replace these special characters with space:
    Replace extra white spaces with single white spaces
    """
    text = re.sub(r"([?.!,])", r" \1 ", text)
    text = re.sub(r'[" "]+', " ", text)
    text = re.sub('[$)\"’°;\'€%:,(/]', '', text)
    text = re.sub('\n', ' ', text)
    text = re.sub('\u200d', ' ', text)
    text = re.sub('\u200c', ' ', text)
    text = re.sub('-', ' ', text)
    text = re.sub('  ', ' ', text)
    text = re.sub('   ', ' ', text)
    text =" ".join(text.split())
    return text

In [14]:
data = pd.read_csv("/content/testData/testEnglish-Bengali.csv")
data.head(10)

,Unnamed: 0,id,english
0,0,177039,current events
1,1,177040,The god Brahma was pleased with her penance bu...
2,2,177041,After feeling stiffness in chest or heaviness ...
3,3,177042,As it dawns on him that the baby is in fact hi...
4,4,177043,Australia produces significant amounts of lent...
5,5,177044,what's with my dinner order
6,6,177045,"In an interview with the New York Times, he cl..."
7,7,177046,The Chief Justice of the Madras High Court is ...
8,8,177047,The funny thing is that near many of the citie...
9,9,177048,It makes their Indian identity obvious .


In [15]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # train
device = "cpu" # inference

In [53]:
# def tokenizer(text):
#   return [tok.lower() for tok in preprocess(text).split()]

# def process(l):
#     engs = []
#     langs = []
#     train_iters = []
#     val_iters = []

#     for e in l:
#         # Define Fields for tokenization and preprocessing
#         lang_field = Field(tokenize=tokenizer, lower=True, init_token="<sos>", eos_token="<eos>")
#         eng_field = Field(tokenize=tokenizer, lower=True, init_token="<sos>", eos_token="<eos>")

#         # Define data fields for loading the dataset
#         datafields = [("english", eng_field), (e, lang_field)]

#         # Load the dataset from a CSV file
#         dataset = TabularDataset(path="/content/{}.csv".format(e), format='csv', skip_header=True, fields=datafields)

#         # Split the dataset into training and validation sets
#         train_data, val_data = dataset.split(split_ratio=0.80)

#         # Build vocabulary for each language from the training data
#         lang_field.build_vocab(train_data, min_freq=1, max_size=50000)
#         eng_field.build_vocab(train_data, min_freq=1, max_size=50000)

#         # Creating the train and validation data iterator for training
#         train_iter, val_iter = BucketIterator.splits(
#             (train_data, val_data),
#             batch_size=8,
#             device=device,
#             sort_key=lambda x: len(getattr(x, e)),
#             sort_within_batch=True)

#         engs.append(eng_field)
#         langs.append(lang_field)
#         train_iters.append(train_iter)
#         val_iters.append(val_iter)

#     return engs, langs, train_iters, val_iters

# engs, langs, train_iters, val_iters = process(l)

### Downloading the params of all the models

In [19]:
class TransformerMT(nn.Module):
    """
    Custom Transformer model for Machine Translation (MT).

    This model consists of an encoder and a decoder, both built using the Transformer architecture.

    Parameters:
    - nhead (int): Number of heads in the multihead attention mechanism.
    - embed_size (int): Dimension of the embedding vector.
    - source_vocab_size (int): Size of the source vocabulary.
    - target_vocab_size (int): Size of the target vocabulary.
    - num_encoder_layers (int): Number of layers in the transformer encoder.
    - num_decoder_layers (int): Number of layers in the transformer decoder.
    - ffnn_size (int, optional): Size of the feedforward neural network inside transformer layers. Default is 512.
    """

    def __init__(self,
                 nhead: int,
                 embed_size: int,
                 source_vocab_size: int,
                 target_vocab_size: int,
                 num_encoder_layers: int,
                 num_decoder_layers: int,
                 ffnn_size:int = 512):

        super(TransformerMT, self).__init__()

        # Define encoder and decoder layers
        encoder_layer = TransformerEncoderLayer(d_model = embed_size, nhead = nhead, dim_feedforward = ffnn_size)
        decoder_layer = TransformerDecoderLayer(d_model = embed_size, nhead = nhead, dim_feedforward = ffnn_size)

        # Initialize transformer encoder, decoder, and final fully connected layer
        self.tf_encoder = TransformerEncoder(encoder_layer, num_layers = num_encoder_layers)
        self.tf_decoder = TransformerDecoder(decoder_layer, num_layers = num_decoder_layers)
        self.fc_layer = nn.Linear(embed_size, target_vocab_size)

        # Embedding layers for tokens and positional information
        self.src_token_embedding = TokenEmbedding(source_vocab_size, embed_size)
        self.tar_token_embedding = TokenEmbedding(target_vocab_size, embed_size)
        self.positional_embedding = SinusoidalEmbedding(embed_size, dropout = DROPOUT_RATE)

    def forward(self,
                source: Tensor,
                target: Tensor,
                source_mask: Tensor,
                target_mask: Tensor,
                source_padding_mask: Tensor,
                target_padding_mask: Tensor,
                memory_key_padding_mask: Tensor):
        """
        Forward pass of the TransformerMT model.

        Parameters:
        - source (torch.Tensor): Source sequence tensor.
        - target (torch.Tensor): Target sequence tensor.
        - source_mask (torch.Tensor): Source sequence mask.
        - target_mask (torch.Tensor): Target sequence mask.
        - source_padding_mask (torch.Tensor): Source padding mask.
        - target_padding_mask (torch.Tensor): Target padding mask.
        - memory_key_padding_mask (torch.Tensor): Memory key padding mask.

        Returns:
        - torch.Tensor: Output tensor after passing through the transformer and the fully connected layer.
        """

        source_embedding = self.positional_embedding(self.src_token_embedding(source))
        target_embedding = self.positional_embedding(self.tar_token_embedding(target))
        memory = self.tf_encoder(source_embedding, source_mask, source_padding_mask)
        outputs = self.tf_decoder(target_embedding,
                                  memory,
                                  target_mask,
                                  None,
                                  target_padding_mask,
                                  memory_key_padding_mask)
        outputs = self.fc_layer(outputs)
        return outputs

    def encode(self, source: Tensor, source_mask: Tensor):
        """
        Encode the source sequence using the transformer encoder.

        Parameters:
        - source (torch.Tensor): Source sequence tensor.
        - source_mask (torch.Tensor): Source sequence mask.

        Returns:
        - torch.Tensor: Encoded memory tensor.
        """
        token_rep = self.src_token_embedding(source)
        positional_rep = self.positional_embedding(token_rep)
        encoder_output = self.tf_encoder(positional_rep, source_mask)
        return encoder_output

    def decode(self, target: Tensor, memory: Tensor, target_mask: Tensor):
        """
        Decode the memory tensor using the transformer decoder.

        Parameters:
        - target (torch.Tensor): Target sequence tensor.
        - memory (torch.Tensor): Encoded memory tensor.
        - target_mask (torch.Tensor): Target sequence mask.

        Returns:
        - torch.Tensor: Decoded tensor.
        """
        token_rep = self.src_token_embedding(target)
        positional_rep = self.positional_embedding(token_rep)
        decoder_output = self.tf_decoder(positional_rep, memory, target_mask)
        return decoder_output



In [20]:
class SinusoidalEmbedding(nn.Module):
    """
    Sinusoidal Positional Encoding for Transformer models.

    The positional encoding module uses sine and cosine functions of different frequencies to
    encode the position of tokens in the sequence.

    Parameters:
    - embed_size (int): Dimension of the embedding vector.
    - dropout (float): Dropout rate for the dropout layer.
    - max_len (int, optional): Maximum length of the sequence. Default is 5000.
    """

    def __init__(self, embed_size, dropout, max_len = 5000):
        super(SinusoidalEmbedding, self).__init__()
        self.dropout = nn.Dropout(p = dropout)

        # Compute the sinusoidal positional encodings
        denom = max_len*2
        pdist = torch.exp(- torch.arange(0, embed_size, 2) * math.log(denom) / embed_size)
        position = torch.arange(0, max_len).reshape(max_len, 1)
        position_embedding = torch.zeros((max_len, embed_size))
        position_embedding[:, 0::2] = torch.sin(position * pdist)
        position_embedding[:, 1::2] = torch.cos(position * pdist)
        position_embedding = position_embedding.unsqueeze(-2)

        # Register the position embeddings so they get saved with the model's state_dict
        self.register_buffer('position_embedding', position_embedding)

    def forward(self, token_embed):
        """
        Forward pass of the SinusoidalEmbedding.

        Parameters:
        - token_embed (torch.Tensor): Token embeddings.

        Returns:
        - torch.Tensor: Token embeddings added with positional encodings.
        """
        outputs = token_embed + self.position_embedding[:token_embed.size(0),:]
        outputs = self.dropout(outputs)
        return outputs


class TokenEmbedding(nn.Module):
    """
    Token Embedding module for Transformer models.

    This module converts token indices into dense vectors of fixed size, embed_size.
    The embeddings are scaled by the square root of their dimensionality.

    Parameters:
    - vocab_size (int): Size of the vocabulary.
    - embed_size (int): Dimension of the embedding vector.
    """

    def __init__(self, vocab_size, embed_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.embed_size = embed_size

    def forward(self, tokens):
        """
        Forward pass of the TokenEmbedding.

        Parameters:
        - tokens (torch.Tensor): Tensor of token indices.

        Returns:
        - torch.Tensor: Scaled token embeddings.
        """
        outputs = self.embedding(tokens.long())
        outputs_scaled = outputs * math.sqrt(self.embed_size)
        return outputs_scaled

In [21]:
def generate_square_subsequent_mask(sz):
    """
    Generate a square mask for the sequence, where the mask indicates subsequent positions.
    This mask is used to ensure that a position cannot attend to subsequent positions in the sequence.

    Parameters:
    - sz (int): Size of the sequence.

    Returns:
    - torch.Tensor: Mask tensor of shape (sz, sz) with 0s in positions that can be attended to and negative infinity elsewhere.
    """

    mask = (torch.triu(torch.ones((sz, sz), device=device)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

def create_mask(source, target, PAD_IDX):
    """
    Create masks for the source and target sequences.

    Parameters:
    - source (torch.Tensor): Source sequence tensor.
    - target (torch.Tensor): Target sequence tensor.

    Returns:
    - tuple: A tuple containing source mask, target mask, source padding mask, and target padding mask.
    """

    source_seq_len = source.shape[0]
    target_seq_len = target.shape[0]
    batch_size = source.shape[1]
    source_mask = torch.zeros((source_seq_len, source_seq_len), device=device).type(torch.bool)
    target_mask = generate_square_subsequent_mask(target_seq_len)
    source_padding_mask = (source == PAD_IDX).transpose(0, 1)
    target_padding_mask = (target == PAD_IDX).transpose(0, 1)
    return source_mask, target_mask, source_padding_mask, target_padding_mask


In [11]:

# \# with open("/content/eng_kannada.pkl", "rb") as file:
# #   eng = pickle.load(file)
# # print(len(eng))

In [24]:
# SOS_IDX = eng.vocab.stoi["<sos>"]
# EOS_IDX = eng.vocab.stoi["<eos>"]

def get_tokens(model, source, source_mask, max_len, start_symbol, SOS_IDX):
    """
    Get token indices from a given source sequence using a trained model.

    Parameters:
    - model (nn.Module): The trained transformer model.
    - source (torch.Tensor): The source sequence tensor.
    - source_mask (torch.Tensor): The mask for the source sequence.
    - max_len (int): Maximum length of the target sequence.
    - start_symbol (int): The starting symbol for the target sequence.

    Returns:
    - result (torch.Tensor): The tensor containing token indices of the target sequence.
    """

    # Move source and its mask to device
    source_mask = source_mask
    source = source

    # Encode the source sequence
    memory = model.encode(source, source_mask)

    # Initialize result tensor with the start symbol
    result = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(device)

    # Decode the memory tensor to get the target sequence
    for index in range(max_len - 1):
        memory = memory.to(device)
        memory_mask = torch.zeros(result.shape[0], memory.shape[0]).to(device).type(torch.bool)
        target_mask = (generate_square_subsequent_mask(result.size(0)).type(torch.bool)).to(device)
        output = model.decode(result, memory, target_mask)
        output = output.transpose(0, 1)

        # Get the next word's probability distribution and find the word with the maximum probability
        probs = model.fc_layer(output[:, -1])
        _, next_word = torch.max(probs, dim = 1)
        next_word = next_word.item()

        # Add the next word to the result
        result = torch.cat([result,torch.ones(1, 1).type_as(source.data).fill_(next_word)], dim=0)

        # Break if the next word is the start of sequence symbol
        if next_word == SOS_IDX:
            break

    return result


def translate(model, source, source_vocab, target_vocab, SOS_IDX, EOS_IDX):
    """
    Translate a given source sequence to a target sequence using a trained model.

    Parameters:
    - model (nn.Module): The trained transformer model.
    - source (str or torch.Tensor): The source sequence (string or tensor).
    - source_vocab (Vocab): Vocabulary object for the source language.
    - target_vocab (Vocab): Vocabulary object for the target language.

    Returns:
    - str: The translated sequence.
    """

    # Set the model to evaluation mode
    model.eval()

    # Convert source string to tensor, if it's a string
    if type(source) == str:
        tokens = [SOS_IDX] + [source_vocab.stoi[tok] for tok in source.split()] + [EOS_IDX]
    else:
        tokens = source
    num_tokens = len(tokens)
    source = (torch.LongTensor(tokens).reshape(num_tokens, 1))
    source_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)

    # Get target token indices from the source tensor
    target_tokens = get_tokens(model, source, source_mask, max_len = num_tokens + 5, start_symbol = SOS_IDX, SOS_IDX = SOS_IDX).flatten()

    # Convert target token indices to string
    return " ".join([target_vocab.itos[token] for token in target_tokens]).replace("<sos>", "").replace("<eos>", "").replace("<unk>", "")


In [25]:
# SRC_VOCAB_SIZE = len(eng.vocab)  # Source vocabulary size (English)
# TAR_VOCAB_SIZE = len(lang.vocab)  # Target vocabulary size (Other language, inferred from 'lang' variable)
EMBEDDING_SIZE = 512  # Size of the embedding vector
NHEAD = 8  # Number of heads in the multihead attention mechanism
FFNN_DIM = 512  # Dimension of the feed-forward neural network inside transformer layers
BATCH_SIZE = 32  # Size of each training batch
NUM_ENCODER_LAYERS = 3  # Number of layers in the transformer encoder
NUM_DECODER_LAYERS = 3  # Number of layers in the transformer decoder
LEARNING_RATE = 0.0001  # Learning rate for the optimizer
DROPOUT_RATE = 0.1  # Dropout rate for the dropout layer
NUM_EPOCHS = 2  # Number of epochs for training
# PAD_IDX = eng.vocab.stoi["<pad>"]  # Index for the padding token
# SOS_IDX = eng.vocab.stoi["<sos>"] # index for start token
# EOD_IDX = eng.vocab.stoi["<eos>"] # index for end token

for idx, e in enumerate(names):
  with open("/content/{}.pkl".format(e), "rb") as vocab_file:
    lang = pickle.load(vocab_file)
    print(len(lang))
  with open("/content/eng_{}.pkl".format(e), "rb") as eng_vocab_file:
    eng = pickle.load(eng_vocab_file)
    print(len(eng))
  SOS_IDX = 2
  EOS_IDX = 3
  print(e)

  # Instantiate the TransformerMT model with the specified configuration
  # if e.lower() == "hindi":
    # break
    # model = TransformerMT(NHEAD,
    #                     EMBEDDING_SIZE,
    #                     len(lang),
    #                     len(lang),
    #                     NUM_ENCODER_LAYERS,
    #                     NUM_DECODER_LAYERS,
    #                     FFNN_DIM)
  # else:
  model = TransformerMT(NHEAD,
                        EMBEDDING_SIZE,
                        len(eng),
                        len(lang),
                        NUM_ENCODER_LAYERS,
                        NUM_DECODER_LAYERS,
                        FFNN_DIM)



  # Define the loss criterion
  # CrossEntropyLoss is used since this is a classification task, and we ignore the loss computed on padding tokens
  criterion = torch.nn.CrossEntropyLoss(ignore_index=0)

  model.load_state_dict(torch.load('/content/{}.pth'.format(e.lower())))
  model.to(device)
  # Define the optimizer to be used for training
  # Adam optimizer is used with specific betas and epsilon values
  optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE, betas=(0.9, 0.98), eps=1e-9)
# Move the model to the specified device (either GPU or CPU)


  # Inference
# Check if the "Translations" directory exists. If not, create it.
  if not os.path.exists("transformer_translations"):
      os.makedirs("transformer_translations")

  # List to store the predicted translations.
  predictions = []

  # Read the test data from the specified CSV file.
  data = pd.read_csv("/content/testData/testEnglish-{}.csv".format(e.title()))
  data = data.iloc[:10, :]

  # Loop through each row (sentence) in the test data.
  for idx, row in data.iterrows():
      # Extract the English sentence.
      en = row["english"]

      # Translate the English sentence to the target language.
      pred = translate(model, en, eng, lang, SOS_IDX, EOS_IDX)

      # Print the translated sentence (optional, can be commented out).
      print(pred)

      # Append the translated sentence to the predictions list.
      predictions.append(pred)

  # Add the predicted translations as a new column to the original dataframe.
  data["translated"] = predictions

  # Drop the unwanted column "Unnamed: 0" (assuming it exists in the CSV).
  data.drop(["Unnamed: 0"], inplace=True, axis=1)

  # Save the dataframe with translations to a new CSV file.
  data.to_csv(os.path.join("transformer_translations", "answer_{}_test.csv".format(e)))

# Evaluate the model on the Bengali test set.



50004
50004
hindi
 और के के के के के के के के के के के के के के के के के के के के के के के के के के के के के के के के के के के के के के के के के के
 के के के के के के के के के के के के       
 और के के के के और और के के के और और और और और हैं  हैं  हैं  है। । ।
 और और और और और और और और और और और और और और और और और और और और और और और और और हैं   
 के के के के के के के के      
 और के के के के के के के के के के के के के के के के के के के के के के के के के के के के के के
 और के के के के के के के के के के के के के के के के के के के के के के के के के के के के के के के के के के के के के के के के
 और के के के के के के के के के के के के के के के के के के के के के के के  
 और के के के के के और के के के और और और और और और और और और और के के के और और और और और के के के के के के के के के के और और और और और और हैं
 और के के के के के के के के के के और और के के के और और और के के के के और और और हैं    
50004
42894
tamil
    மற்றும் மற்றும் மற்றும் மற்றும் . மற்றும் . மற்றும் . மற்றும் . . . . . . . . . . . . 

In [26]:
DATA_DIR = "/content/transformer_translations"
RESULTS = "answer.csv"
VALIDATION_FILE = "/content/test_data.json"

if os.path.exists(os.path.join(DATA_DIR, RESULTS)):
    os.remove(os.path.join(DATA_DIR,RESULTS))
val_ids = []

with open(VALIDATION_FILE, "r") as file:
    val_data = json.load(file)
    for lang_type, lang_data in val_data.items():
        print(lang_type)
        for data_type, data_entries in lang_data.items():
            ids, en_sns = [], []
            for ent_id, ent_data in data_entries.items():
                val_ids.append(int(ent_id))

print(len(val_ids))
# print(val_ids)

# Reading all the dataframes
files = os.listdir(DATA_DIR)
files = [file for file in files if file.endswith(".csv")]
print(files)
data = [pd.read_csv(os.path.join(DATA_DIR, file)) for file in files]
res = pd.concat(data, axis=0, ignore_index=False)
res.drop(["Unnamed: 0", "english"], axis=1, inplace=True)
res = res.rename(columns={"id": "ID", "translated": "Translated"})
res_sorted = res[res["ID"].isin(val_ids)].sort_values(by="ID")
res_sorted.to_csv(os.path.join(DATA_DIR, RESULTS), sep="\t", index=False, quoting=csv.QUOTE_NONNUMERIC, quotechar='"', escapechar='\\')


English-Bengali
English-Gujarati
English-Hindi
English-Kannada
English-Malayalam
English-Tamil
English-Telgu
114647
['answer_malayalam_test.csv', 'answer_bengali_test.csv', 'answer_gujarati_test.csv', 'answer_kannada_test.csv', 'answer_tamil_test.csv', 'answer_telgu_test.csv', 'answer_hindi_test.csv']
